In [1]:
import pandas as pd
import numpy as np

import sys, os
sys.path.append(os.path.abspath("../src"))

import dataanalyse
import importlib
importlib.reload(dataanalyse)


<module 'dataanalyse' from 'c:\\Users\\idunn\\OneDrive - NTNU\\TDT4114\\Mappe\\src\\dataanalyse.py'>

In [28]:
#Temperaturdata
statistikk_temp = dataanalyse.analyser_fil("../data/temp_gloshaugen_historisk_renset_ 50.csv",datokolonne="tidspunkt",groupby="måned")

for navn, df in statistikk_temp.items():
    print(f"\nSamlet statistikk for: {navn}")

    if 'år' in df.columns:
        display(df.round(2))

    if 'år' in df.columns and 'måned' in df.columns:
        print(f"Årlig gjennomsnitt (kun hele år) for: {navn}")

        # Finn hvor mange måneder hvert år har og tar kun med de som har 12 måneder
        antall_mnd = df.groupby('år')['måned'].count().reset_index(name='antall_måneder')
        hele_år = antall_mnd[antall_mnd['antall_måneder'] == 12]['år']

        df_hele_år = df[df['år'].isin(hele_år)]
        årlig_snitt = df_hele_år.groupby('år')['gjennomsnitt'].mean().reset_index(name='årsgjennomsnitt')

        display(årlig_snitt.round(2))


Analyse av:../data/temp_gloshaugen_historisk_renset_ 50.csv
-------------------------------------------------

Samlet statistikk for: temperatur


,år,måned,gjennomsnitt,median,standardavvik
0,1976,1,-4.55,-4.10,4.65
1,1976,2,-2.26,-1.90,4.53
2,1976,3,-1.20,-1.30,4.40
3,1976,4,5.57,5.55,3.81
4,1976,5,12.56,12.50,4.65
...,...,...,...,...,...
587,2024,12,1.42,1.70,3.46
588,2025,1,-1.39,-0.90,4.10
589,2025,2,-0.99,-0.85,3.84
590,2025,3,4.08,4.15,4.15


Årlig gjennomsnitt (kun hele år) for: temperatur


,år,årsgjennomsnitt
0,1976,6.46
1,1977,6.26
2,1978,5.75
3,1979,5.02
4,1980,5.73
5,1981,5.44
6,1982,6.53
7,1983,7.30
8,1984,6.96
9,1985,4.67


In [ ]:
# Beregn endring i gjennomsnittstemperatur fra første til siste år
første = årlig_snitt.iloc[0]
siste = årlig_snitt.iloc[-1]
endring = siste['årsgjennomsnitt'] - første['årsgjennomsnitt']
prosent = (endring / abs(første['årsgjennomsnitt'])) * 100

print(f"Endring i temperatur fra {int(første['år'])} til {int(siste['år'])}:")
print(f"{endring:.2f} °C ({prosent:.2f} %)")



Endring i temperatur fra 1976 til 2024:
1.23 °C (19.09 %)


In [30]:
varmest = årlig_snitt.loc[årlig_snitt['årsgjennomsnitt'].idxmax()]
kaldest = årlig_snitt.loc[årlig_snitt['årsgjennomsnitt'].idxmin()]

print(f"Varmeste år: {int(varmest['år'])} med {varmest['årsgjennomsnitt']:.2f} °C")
print(f"Kaldeste år: {int(kaldest['år'])} med {kaldest['årsgjennomsnitt']:.2f} °C")


Varmeste år: 2020 med 8.76 °C
Kaldeste år: 1985 med 4.67 °C


In [ ]:
# Gjenomsnittet for hvert tiår
årlig_snitt['tiår'] = (årlig_snitt['år'] // 10) * 10
tiårs_snitt = årlig_snitt.groupby('tiår')['årsgjennomsnitt'].mean().reset_index()

display(tiårs_snitt.round(2))


,tiår,årsgjennomsnitt
0,1970,5.87
1,1980,6.21
2,1990,6.60
3,2000,7.11
4,2010,7.11
5,2020,7.69


In [ ]:
# Temperaturavvik per år
total_snitt = årlig_snitt['årsgjennomsnitt'].mean()
årlig_snitt['avvik'] = årlig_snitt['årsgjennomsnitt'] - total_snitt

display(årlig_snitt[['år', 'årsgjennomsnitt', 'avvik']].round(2))


,år,årsgjennomsnitt,avvik
0,1976,6.46,-0.33
1,1977,6.26,-0.52
2,1978,5.75,-1.03
3,1979,5.02,-1.76
4,1980,5.73,-1.05
5,1981,5.44,-1.34
6,1982,6.53,-0.25
7,1983,7.30,0.52
8,1984,6.96,0.18
9,1985,4.67,-2.12


In [13]:
#Klimasaddutslipp
statistikk_klimagass = dataanalyse.analyser_fil("../data/klimagassutslipp_renset.csv",sep=",",datokolonne="år",groupby="år")

for navn, df in statistikk_klimagass.items():
    print(f"\nStatistikk: {navn}")
    
    if 'kilde (aktivitet)' in df.columns:
        for kilde, kilde_df in df.groupby('kilde (aktivitet)'):
            print(f"Kilde: {kilde}")
            display(kilde_df.round(2))
    else:
        display(df.round(2))


Analyse av:../data/klimagassutslipp_renset.csv
-------------------------------------------------

Statistikk: utslipp_til_luft_(1_000_tonn_co2-ekvivalenter,_ar5)


,år,gjennomsnitt,median,standardavvik
0,1990,11410.67,5298.0,15910.65
1,1991,10885.89,5109.0,15120.88
2,1992,10544.44,5461.0,14515.41
3,1993,10973.11,5664.0,15140.90
4,1994,11407.00,5858.0,15777.98
5,1995,11498.00,6554.0,15834.17
6,1996,12142.56,6960.0,16700.55
7,1997,12137.44,7246.0,16703.36
8,1998,12154.44,7463.0,16749.45
9,1999,12377.22,7526.0,17029.17


In [37]:
# Samlet statestikk for klimagassutslipp 

for navn, df in statistikk_klimagass.items():
    print(f"Samlet statistikk for: {navn}")

    # Fjerner 'Alle kilder'
    if 'år' in df.columns and 'kilde (aktivitet)' in df.columns:
        df['kilde (aktivitet)'] = df['kilde (aktivitet)'].astype(str).str.strip()
        df_uten_alle = df[~df['kilde (aktivitet)'].str.lower().str.contains("alle kilder")]

        # Vis gjennomsnitt per år (uten "Alle kilder")
        samlet = df_uten_alle.groupby('år').mean(numeric_only=True).reset_index()
        display(samlet.round(2))

        # Beregn snitt per kilde basert på gjennomsnitt (regnet ut over)
        if 'gjennomsnitt' in df.columns:
            snitt_per_kilde = df_uten_alle.groupby('kilde (aktivitet)')[['gjennomsnitt']].mean().reset_index()
            høyest = snitt_per_kilde.sort_values(by='gjennomsnitt', ascending=False).head(1)

            print("Kilde med høyest gjennomsnitt utslipp:")
            print(høyest.round(2))
        else:
            print("Fant ikke gjennomsnitt-kolonnen")
    else:
        print("Mangler kolonne('år' og/eller 'kilde (aktivitet)')")


Samlet statistikk for: utslipp til luft (1 000 tonn co2-ekvivalenter, ar5)


,år,gjennomsnitt,median
0,1990,6418.50,6418.50
1,1991,6123.25,6123.25
2,1992,5931.25,5931.25
3,1993,6172.38,6172.38
4,1994,6416.50,6416.50
5,1995,6467.62,6467.62
6,1996,6830.12,6830.12
7,1997,6827.25,6827.25
8,1998,6837.00,6837.00
9,1999,6962.12,6962.12


Kilde med høyest gjennomsnitt utslipp:
        kilde (aktivitet)  gjennomsnitt
1  2 Industri og bergverk       14117.5


In [39]:
# Årlig endring (absolutt og prosentvis):
df['absolutt_endring'] = df['gjennomsnitt'].diff()
df['prosent_endring'] = df['gjennomsnitt'].pct_change() * 100
display(df[['år', 'gjennomsnitt', 'absolutt_endring', 'prosent_endring']].round(2))

,år,gjennomsnitt,absolutt_endring,prosent_endring
0,1990,51348.0,NaN,NaN
1,1991,48987.0,-2361.0,-4.60
2,1992,47450.0,-1537.0,-3.14
3,1993,49379.0,1929.0,4.07
4,1994,51331.0,1952.0,3.95
...,...,...,...,...
301,2019,2398.0,-105.0,-4.19
302,2020,2341.0,-57.0,-2.38
303,2021,2294.0,-47.0,-2.01
304,2022,2242.0,-52.0,-2.27


In [59]:
# Året med høyest og lavest utslipp
df_totalt = df[df['kilde (aktivitet)'].str.lower().str.contains("alle kilder")].copy()

høyest = df_totalt.loc[df_totalt['gjennomsnitt'].idxmax()]
lavest = df_totalt.loc[df_totalt['gjennomsnitt'].idxmin()]

print("År med høyest totalutslipp:")
display(pd.DataFrame([høyest]).round(2))

print("År med lavest totalutslipp:")
display(pd.DataFrame([lavest]).round(2))


År med høyest totalutslipp:


,kilde (aktivitet),år,gjennomsnitt,median,absolutt_endring,prosent_endring
17,0 Alle kilder,2007,56548.0,56548.0,1678.0,3.06


År med lavest totalutslipp:


,kilde (aktivitet),år,gjennomsnitt,median,absolutt_endring,prosent_endring
33,0 Alle kilder,2023,46654.0,46654.0,-2319.0,-4.74


In [67]:
# Gjennomsnitt gruppert etter ti-år
df_totalt['tiår'] = (df_totalt['år'] // 10) * 10
trend = df_totalt.groupby('tiår')['gjennomsnitt'].mean().reset_index()
display(trend.round(2))


,tiår,gjennomsnitt
0,1990,51988.90
1,2000,55203.50
2,2010,53432.20
3,2020,48612.75


In [2]:
statistikk_luft = dataanalyse.analyser_fil("../data/gyldig_historisk_luftkvalitet.csv", datokolonne="tid", groupby="år")

for stoff, df in statistikk_luft.items():
    print(f"Statistikk: {stoff}")
    display(df[['år', 'gjennomsnitt']].round(2))



Analyse av:../data/gyldig_historisk_luftkvalitet.csv
-------------------------------------------------
Statistikk: elgeseter_no2_ugm3_day


c:\Users\idunn\OneDrive - NTNU\TDT4114\Mappe\src\dataanalyse.py:27: UserWarning: Parsing dates in %d.%m.%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df[datokolonne] = pd.to_datetime(df[datokolonne], errors='coerce')


,år,gjennomsnitt
0,2003,48.27
1,2004,59.01
2,2005,52.45
3,2006,56.64
4,2007,63.51
5,2008,53.38
6,2009,52.99
7,2010,51.11
8,2011,45.63
9,2012,44.44


Statistikk: elgeseter_pm10_ugm3_day


,år,gjennomsnitt
0,2003,29.01
1,2004,33.14
2,2005,35.21
3,2006,42.77
4,2007,30.01
5,2008,26.56
6,2009,29.01
7,2010,26.96
8,2011,31.35
9,2012,29.73


Statistikk: elgeseter_pm2.5_ugm3_day


,år,gjennomsnitt
0,2003,14.90
1,2004,14.97
2,2005,15.30
3,2006,15.28
4,2007,12.50
5,2008,10.53
6,2009,10.73
7,2010,11.25
8,2011,10.33
9,2012,9.99


In [3]:
df_luft = pd.read_csv("../data/gyldig_historisk_luftkvalitet.csv", sep=",")

df_luft.columns = df_luft.columns.str.strip().str.lower()
df_luft.columns = df_luft.columns.str.replace(" ", "_").str.replace("µg/m³", "ugm3").str.replace("ug/m3", "ugm3")

df_luft["tid"] = pd.to_datetime(df_luft["tid"], errors="coerce")
df_luft["dato"] = df_luft["tid"].dt.date
df_luft["år"] = df_luft["tid"].dt.year
df_luft["måned"] = df_luft["tid"].dt.month
df_luft["time"] = df_luft["tid"].dt.hour

målinger = [kol for kol in df_luft.columns if "ugm3" in kol]

C:\Users\idunn\AppData\Local\Temp\ipykernel_16964\1563204585.py:6: UserWarning: Parsing dates in %d.%m.%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_luft["tid"] = pd.to_datetime(df_luft["tid"], errors="coerce")


In [4]:
# Analyser hver måling separat
for stoff in målinger:
    print("-------------------------------------------------------------")
    print(f"               Analyse av: {stoff}")
    print("-------------------------------------------------------------")

    # Gjennomsnitt (dag) 
    # Kan brukes hvis relevant i vidre analyse, men blir ikke printet ut her 
    daglig = df_luft.groupby("dato")[stoff].mean().reset_index(name="daglig_snitt")

    # Gjennomsnitt (år)
    årlig = df_luft.groupby("år")[stoff].mean().reset_index(name="årssnitt")
    print("Årlig Gjennomsnitt:")
    display(årlig.round(2))

    # Gjennomsnitt (måned)
    månedlig = df_luft.groupby("måned")[stoff].mean().reset_index(name="månedssnitt")
    print("Månedlig gjennomsnitt:")
    display(månedlig.round(2))

    # Luftkvalitet snitt pr. klokkeslett
    rush = df_luft.groupby("time")[stoff].mean().reset_index(name="snitt_per_klokkeslett")
    print(f"Når på døgnet det er mest {stoff} :")
    display(rush.round(2))

    # Ekstremal verdier 
    maks_rad = df_luft.loc[df_luft[stoff].idxmax()]
    min_rad = df_luft.loc[df_luft[stoff].idxmin()]
    print(f"Ekstremer for {stoff}:")
    print(f"Maks: {maks_rad[stoff]:.2f} µg/m³ @ {maks_rad['tid']}")
    print(f"Min: {min_rad[stoff]:.2f} µg/m³ @ {min_rad['tid']}")

-------------------------------------------------------------
               Analyse av: elgeseter_no2_ugm3_day
-------------------------------------------------------------
Årlig Gjennomsnitt:


,år,årssnitt
0,2003,48.27
1,2004,59.01
2,2005,52.45
3,2006,56.64
4,2007,63.51
5,2008,53.38
6,2009,52.99
7,2010,51.11
8,2011,45.63
9,2012,44.44


Månedlig gjennomsnitt:


,måned,månedssnitt
0,1,51.46
1,2,50.60
2,3,42.90
3,4,35.34
4,5,28.51
5,6,24.87
6,7,22.38
7,8,27.36
8,9,28.73
9,10,36.07


Når på døgnet det er mest elgeseter_no2_ugm3_day :


...

Månedlig gjennomsnitt:


,måned,månedssnitt
0,1,11.33
1,2,9.27
2,3,7.82
3,4,7.62
4,5,6.37
5,6,6.16
6,7,5.67
7,8,5.73
8,9,6.12
9,10,8.56


Når på døgnet det er mest elgeseter_pm2.5_ugm3_day :


,time,snitt_per_klokkeslett
0,0,8.24


Ekstremer for elgeseter_pm2.5_ugm3_day:
Maks: 66.62 µg/m³ @ 2008-01-01 00:00:00
Min: 0.00 µg/m³ @ 2014-12-23 00:00:00
